In [80]:
import requests
import pandas as pd
from time import sleep
from IPython.display import clear_output

In [82]:
def getData(continueVariable, startDate, endDate):
    global S
    # continueVariable is a key containing the next set of records
    if(continueVariable != ''):
        PARAMS = {
            "action": "query",
            "format": "json",
            "list": "logevents",
            "lelimit": "500",
            "letype":"block",
            "lestart":startDate,
            "leend":endDate,
            "ledir":"newer",
            "lecontinue" : continueVariable
        }
        
        R = S.get(url=URL, params=PARAMS)
        DATA = R.json()
        if 'continue' in DATA:
            continueVariable = DATA['continue']['lecontinue']
        else:
            continueVariable = ''
        LOGS = DATA["query"]["logevents"]
        return LOGS, continueVariable
    
    else:
        # If end of records then return custom -1 and empty string
        return -1,''
        
        

In [83]:
def updateCSV(year, data):
    # Make data frame of above data
    df = pd.DataFrame.from_records(data)
    
    # append data frame to CSV file
    df.to_csv(str(year)+'.csv', mode='a', index=False, header=False)


In [84]:
# Send first initial request to get the first continueVariable
def firstCall(startDate, endDate, year):
  global S
  global URL

  PARAMS = {
      "action": "query",
      "format": "json",
      "list": "logevents",
      "lelimit": "500",
      "letype":"block",
      "lestart":startDate,
      "leend":endDate,
      "ledir":"newer"
  }

  R = S.get(url=URL, params=PARAMS)

  DATA = R.json()

  LOGS = DATA["query"]["logevents"]
  continueVariable = DATA['continue']['lecontinue']
  # Initialize to a local csv
  df = pd.DataFrame.from_records(LOGS)
  df.to_csv(str(year)+'.csv', mode='a', index=False)
  return LOGS, continueVariable

In [91]:
def getAllYearsData(years):
  for year in date_range:
    startDate = str(year)+'-01-01T00:00:00Z'
    endDate = str(year+1)+'-01-01T00:00:00Z'

    LOGS, continueVariable = firstCall(startDate, endDate, year)

    count = len(LOGS)
    while(continueVariable != ''):
        newLogs, continueVariable = getData(continueVariable, startDate, endDate)
        count+= len(newLogs)
        if(continueVariable == ''):
            print(f'{year} DONE! Total Records fetched {count}')
            break
        updateCSV(year,newLogs)


In [81]:
date_range = [*range (2004, 2022, 1)]
S = requests.Session()
URL = "https://en.wikipedia.org/w/api.php"

In [ ]:
getAllYearsData(date_range)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

path = '/content/drive/MyDrive/<ADD_YOUR_YEARLY_DATA_PATH_HERE>'
df = pd.DataFrame()
for data in date_range:
  csvData = pd.read_csv(f'{path}/{data}.csv', on_bad_lines='skip')
  print(data)
  if data in [2007, 2008, 2009]:
    csvData = csvData.drop('actionhidden', axis=1)
    csvData = csvData.drop('suppressed', axis=1)
  data = pd.DataFrame(csvData)
  frames = [df, data]

  df = pd.concat(frames)

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4227547 entries, 0 to 736167
Data columns (total 11 columns):
 #   Column     Dtype  
---  ------     -----  
 0   logid      int64  
 1   ns         float64
 2   title      object 
 3   pageid     object 
 4   logpage    object 
 5   params     object 
 6   type       object 
 7   action     object 
 8   user       object 
 9   timestamp  object 
 10  comment    object 
dtypes: float64(1), int64(1), object(9)
memory usage: 387.0+ MB
